In [1]:
# import libraries 
import pandas as pd
import csv
import numpy as np
from pathlib import Path
from basketball_reference_scraper.seasons import get_schedule
from itertools import product
from tkinter import *
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')


## Data cleaning and preparation

In [2]:
# get 2008-2020, 2020-2021 nba playoffs schedule 
playoff2020 = get_schedule(2020, playoffs = True)
playoff2019 = get_schedule(2019, playoffs = True)
playoff2018 = get_schedule(2018, playoffs = True)
playoff2017 = get_schedule(2017, playoffs = True)
playoff2016 = get_schedule(2016, playoffs = True)
playoff2015 = get_schedule(2015, playoffs = True)
playoff2014 = get_schedule(2014, playoffs = True)
playoff2013 = get_schedule(2013, playoffs = True)
playoff2012 = get_schedule(2012, playoffs = True)
playoff2011 = get_schedule(2011, playoffs = True)
playoff2010 = get_schedule(2010, playoffs = True)
playoff2009 = get_schedule(2009, playoffs = True)
playoff2008 = get_schedule(2008, playoffs = True)
# np.savetxt('2019playoff.csv', playoff2019, delimiter =",", fmt ="% s")
# np.savetxt('2020playoff.csv', playoff2020, delimiter =",", fmt ="% s")

In [3]:
# make playoff results df
playoff_list= [playoff2008,playoff2009,playoff2010,playoff2011,playoff2012,playoff2013,playoff2014,
               playoff2015,playoff2016,playoff2017,playoff2018,playoff2019,playoff2020]
playoffsresults= pd.concat(playoff_list, join = "inner")
playoffsresults.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
1231,2008-04-19,Washington Wizards,86,Cleveland Cavaliers,93
1232,2008-04-19,Phoenix Suns,115,San Antonio Spurs,117
1233,2008-04-19,Dallas Mavericks,92,New Orleans Hornets,104
1234,2008-04-19,Utah Jazz,93,Houston Rockets,82
1235,2008-04-20,Toronto Raptors,100,Orlando Magic,114


In [4]:
# load 2008-2020 playoffs data
csv_updatedmain = Path('updatedmain.csv')
df_playoffsstats = pd.read_csv(csv_updatedmain)
df_playoffsstats.tail(5)

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
1080,Finals,2020-10-02,(2019-20),MIA,LAL,114,124,10,39.5,84.4,...,10.66,35.07,45.73,25.39,8.62,6.59,15.17,20.72,113.44,-600
1081,Finals,2020-10-04,(2019-20),LAL,MIA,104,115,11,42.3,88.3,...,8.49,35.88,44.37,25.92,7.49,4.55,14.90,20.56,112.04,350
1082,Finals,2020-10-06,(2019-20),LAL,MIA,102,96,-6,42.3,88.3,...,8.49,35.88,44.37,25.92,7.49,4.55,14.90,20.56,112.04,270
1083,Finals,2020-10-09,(2019-20),MIA,LAL,111,108,-3,39.5,84.4,...,10.66,35.07,45.73,25.39,8.62,6.59,15.17,20.72,113.44,-340
1084,Finals,2020-10-11,(2019-20),LAL,MIA,106,93,-13,42.3,88.3,...,8.49,35.88,44.37,25.92,7.49,4.55,14.90,20.56,112.04,210


In [5]:
# get betting data, filter out Visitor team data & repeat.  
csv_betting = Path('betting.csv')
df_betting = pd.read_csv(csv_betting)
df_betting = df_betting[df_betting.index%2==1]
df_betting.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
1,1222,502,H,Brooklyn,40,23,36,26,125,5.5,7.5,-320,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1222,504,H,LALakers,19,35,24,31,109,2.5,2,-135,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1223,552,H,Cleveland,24,41,28,28,121,1,217,120,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1223,554,H,Indiana,35,26,27,33,121,8.5,7.5,-330,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1223,556,H,Orlando,27,29,23,34,113,220.5,219,170,106.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# create series matches unique id 

df_playoffsstats['matches'] = df_playoffsstats['VTeam']+df_playoffsstats['HTeam']
u_id = []
for i, r in df_playoffsstats.iterrows():
    if r.loc['matches'][0:3] < r.loc['matches'][3:7]:
        u_id_ = r.matches[0:3] + r.matches[3:7]
        u_id.append(u_id_)
    elif r.loc['matches'][0:3] > r.loc['matches'][3:7]:
        u_id_ = r.matches[3:7] + r.matches[0:3]
        u_id.append(u_id_)
#         print(False, r['u_id'])
df_playoffsstats['u_id'] = u_id
# create 'winner' column in the dataframe
def ss(df_playoffsstats):
    if df_playoffsstats['Mar.'] < 0 :
        return df_playoffsstats['HTeam']
    else:
        return df_playoffsstats['VTeam']
df_playoffsstats['winner'] = df_playoffsstats.apply(ss,axis=1)

df_playoffsstats.head()

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,matches,u_id,winner
0,Playoffs,2008-04-19,(2007-08),WAS,CLE,86,93,7,36.41,81.65,...,20.00,7.06,5.21,13.96,20.88,96.38,-175,WASCLE,CLEWAS,WAS
1,Playoffs,2008-04-19,(2007-08),PHO,SAS,115,117,2,41.37,82.71,...,20.95,6.35,4.11,12.62,18.74,95.37,-200,PHOSAS,PHOSAS,PHO
2,Playoffs,2008-04-19,(2007-08),DAL,NOH,92,104,12,36.89,79.45,...,21.77,7.77,3.91,11.88,18.67,100.85,-205,DALNOH,DALNOH,DAL
3,Playoffs,2008-04-19,(2007-08),UTA,HOU,93,82,-11,39.99,80.39,...,21.43,7.29,5.12,13.72,19.62,96.72,-110,UTAHOU,HOUUTA,HOU
4,Playoffs,2008-04-20,(2007-08),TOR,ORL,100,114,14,38.39,82.02,...,20.80,6.34,4.06,14.30,20.59,104.48,-265,TORORL,ORLTOR,TOR


In [7]:
# sort dataframe by u_id and time
df_playoffsstats['Date'] = pd.to_datetime(df_playoffsstats['Date']).dt.date
df_playoffsstats = df_playoffsstats.sort_values(["u_id", "Date"])
df_playoffsstats.reset_index()
df_playoffsstats['match'] = df_playoffsstats.u_id.eq(df_playoffsstats.u_id.shift())

In [8]:
# get series score
team_one = []
team_two = []

for i, r in df_playoffsstats.iterrows():
    if r['match'] == False:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = 1
            team_two_ = 0
            team_one.append(team_one_)
            team_two.append(team_two_)
            
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = 0
            team_two_ = 1
            team_one.append(team_one_)
            team_two.append(team_two_)
               
    elif r.loc['match'] == True:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = team_one[-1] + 1
            team_two_ = team_two[-1]
            team_one.append(team_one_)
            team_two.append(team_two_)
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = team_one[-1]
            team_two_ = team_two[-1] + 1
            team_one.append(team_one_)
            team_two.append(team_two_)
            
df_playoffsstats['team_one'] = team_one
df_playoffsstats['team_two'] = team_two

df_playoffsstats.head(5)

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPTO,OPPF,OPPTS,H Betting Line,matches,u_id,winner,match,team_one,team_two
7,Playoffs,2008-04-20,(2007-08),ATL,BOS,81,104,23,36.28,79.90,...,15.20,22.73,100.55,-3000,ATLBOS,ATLBOS,ATL,False,1,0
14,Playoffs,2008-04-23,(2007-08),ATL,BOS,77,96,19,36.28,79.90,...,15.20,22.73,100.55,-3000,ATLBOS,ATLBOS,ATL,True,2,0
24,Playoffs,2008-04-26,(2007-08),BOS,ATL,93,102,9,36.41,76.66,...,14.91,20.40,98.22,370,BOSATL,ATLBOS,BOS,True,2,1
31,Playoffs,2008-04-28,(2007-08),BOS,ATL,92,97,5,36.41,76.66,...,14.91,20.40,98.22,380,BOSATL,ATLBOS,BOS,True,2,2
38,Playoffs,2008-04-30,(2007-08),ATL,BOS,85,110,25,36.28,79.90,...,15.20,22.73,100.55,-230,ATLBOS,ATLBOS,ATL,True,3,2


In [9]:
# Make a result column, set home team win to 1, away team win to 0.
results = []
def mr(df_playoffsstats):
    if df_playoffsstats['Mar.'] > 0 :
        return 1
    else:
        return 0
df_playoffsstats['results'] = df_playoffsstats.apply(mr,axis=1)

df_playoffsstats.head(50)
        

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPPF,OPPTS,H Betting Line,matches,u_id,winner,match,team_one,team_two,results
7,Playoffs,2008-04-20,(2007-08),ATL,BOS,81,104,23,36.28,79.90,...,22.73,100.55,-3000,ATLBOS,ATLBOS,ATL,False,1,0,1
14,Playoffs,2008-04-23,(2007-08),ATL,BOS,77,96,19,36.28,79.90,...,22.73,100.55,-3000,ATLBOS,ATLBOS,ATL,True,2,0,1
24,Playoffs,2008-04-26,(2007-08),BOS,ATL,93,102,9,36.41,76.66,...,20.40,98.22,370,BOSATL,ATLBOS,BOS,True,2,1,1
31,Playoffs,2008-04-28,(2007-08),BOS,ATL,92,97,5,36.41,76.66,...,20.40,98.22,380,BOSATL,ATLBOS,BOS,True,2,2,1
38,Playoffs,2008-04-30,(2007-08),ATL,BOS,85,110,25,36.28,79.90,...,22.73,100.55,-230,ATLBOS,ATLBOS,ATL,True,3,2,1
41,Playoffs,2008-05-02,(2007-08),BOS,ATL,100,103,3,36.41,76.66,...,20.40,98.22,370,BOSATL,ATLBOS,BOS,True,3,3,1
45,Playoffs,2008-05-04,(2007-08),ATL,BOS,65,99,34,36.28,79.90,...,22.73,100.55,-2100,ATLBOS,ATLBOS,ATL,True,4,3,1
340,Playoffs,2012-04-29,(2011-12),BOS,ATL,74,83,9,35.47,77.06,...,17.85,96.59,-110,BOSATL,ATLBOS,BOS,True,4,4,1
345,Playoffs,2012-05-01,(2011-12),BOS,ATL,87,80,-7,35.47,77.06,...,17.85,96.59,-210,BOSATL,ATLBOS,ATL,True,5,4,0
353,Playoffs,2012-05-04,(2011-12),ATL,BOS,84,90,6,36.80,81.03,...,19.92,91.85,-450,ATLBOS,ATLBOS,ATL,True,6,4,1


## Random Forest Training and Testing

In [10]:
# Set x variable list of features 
df_playoffsstats_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line']

In [11]:
# Construct the X_train and y_train datasets
df_playoffsstats_x_train = df_playoffsstats[df_playoffsstats_x_var_list]
df_playoffsstats_y_train = df_playoffsstats['results']

In [12]:
# Set path to CSV and read 2021 playoffs data in CSV
csv_path = Path('2021playoff.csv')
df_2021 = pd.read_csv(csv_path)
df_2021.set_index('Time', inplace=True)
df_2021.head()

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
Time,,,,,,,,,,,,,,,,,,,,,
2021-05-22,(2020-21),MIA,MIL,107,109,2,39.22,83.74,0.47,12.94,...,10.3,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1,-220
2021-05-22,(2020-21),DAL,LAC,113,103,-10,41.08,87.32,0.47,13.81,...,9.4,34.7,44.2,24.4,7.1,4.1,13.2,19.2,114.0,-250
2021-05-22,(2020-21),BOS,BRO,93,104,11,41.46,88.90,0.47,13.60,...,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6,-380
2021-05-22,(2020-21),POR,DEN,123,109,-14,41.29,91.08,0.45,15.72,...,10.5,33.9,44.4,26.8,8.1,4.5,13.5,19.1,115.1,-120
2021-05-23,(2020-21),WAS,PHI,118,125,7,43.17,90.93,0.47,10.18,...,10.0,35.0,45.1,23.7,9.1,6.2,14.4,20.2,113.6,-360


In [13]:
# Make a result column, set home team win to 1, away team win to 0.
results = []
def mr(df_2021):
    if df_2021['Mar.'] > 0 :
        return 1
    else:
        return 0
df_2021['results'] = df_2021.apply(mr,axis=1)

df_2021.head()
        

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,results
Time,,,,,,,,,,,,,,,,,,,,,
2021-05-22,(2020-21),MIA,MIL,107,109,2,39.22,83.74,0.47,12.94,...,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1,-220,1
2021-05-22,(2020-21),DAL,LAC,113,103,-10,41.08,87.32,0.47,13.81,...,34.7,44.2,24.4,7.1,4.1,13.2,19.2,114.0,-250,0
2021-05-22,(2020-21),BOS,BRO,93,104,11,41.46,88.90,0.47,13.60,...,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6,-380,1
2021-05-22,(2020-21),POR,DEN,123,109,-14,41.29,91.08,0.45,15.72,...,33.9,44.4,26.8,8.1,4.5,13.5,19.1,115.1,-120,0
2021-05-23,(2020-21),WAS,PHI,118,125,7,43.17,90.93,0.47,10.18,...,35.0,45.1,23.7,9.1,6.2,14.4,20.2,113.6,-360,1


In [14]:
# Set 20x variable list of features
df_2021_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line']

In [15]:
# Construct testing start and end dates
playoff2021_testing_start =  '2021-05-22'
playoff2021_testing_end = '2021-07-03'

In [80]:
# Construct the X test and y test datasets
df_2021_test_X = df_2021[df_2021_x_var_list][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X = df_2021_test_X.reset_index().drop(columns = 'Time')
df_2021_test_y = df_2021['results'][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X.head()

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,39.22,83.74,0.47,12.94,36.19,0.36,16.68,21.11,0.79,8.04,...,10.3,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1,-220
1,41.08,87.32,0.47,13.81,38.11,0.36,16.47,21.17,0.78,9.13,...,9.4,34.7,44.2,24.4,7.1,4.1,13.2,19.2,114.0,-250
2,41.46,88.90,0.47,13.60,36.36,0.37,16.11,20.78,0.78,10.63,...,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6,-380
3,41.29,91.08,0.45,15.72,40.82,0.39,17.81,21.64,0.82,10.64,...,10.5,33.9,44.4,26.8,8.1,4.5,13.5,19.1,115.1,-120
4,43.17,90.93,0.47,10.18,29.00,0.35,20.13,26.17,0.77,9.68,...,10.0,35.0,45.1,23.7,9.1,6.2,14.4,20.2,113.6,-360


In [17]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_playoffsstats_forest_model = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42)
df_playoffsstats_forest_model.fit(df_playoffsstats_x_train, df_playoffsstats_y_train)

RandomForestClassifier(max_depth=4, random_state=42)

In [18]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions = df_playoffsstats_forest_model.predict(df_2021_test_X)

In [19]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results = df_2021_test_y.to_frame()
playoff2021_results["Predicted Mar."] = playoff2021_predictions
playoff2021_results.tail(50)

,results,Predicted Mar.
Time,,
2021-05-30,0,0
2021-05-31,1,0
2021-05-31,0,0
2021-06-01,1,1
2021-06-01,1,1
2021-06-01,1,1
2021-06-02,1,1
2021-06-02,0,0
2021-06-02,1,1


In [20]:
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions))

0.6708860759493671


## Create GUI for our predictor

In [21]:
# load team stats data
teamstats2021_path = Path('2021teamseasonstats.csv')
df_team_2021 = pd.read_csv(teamstats2021_path)
df_team_2021 = df_team_2021.set_index('Team')
df_team_2021.head()

,League,Type,Season,Team/Opponent,Champion,Franchise,FGM,FGA,FG%,3FGM,...,FT%,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS
Team,,,,,,,,,,,,,,,,,,,,,
ATL,NBA,Regular Season,(2020-21),Team Stats,NaN,Atlanta Hawks,40.8,87.2,0.47,12.4,...,0.81,10.6,35.1,45.6,24.1,7.0,4.8,13.2,19.3,113.7
BOS,NBA,Regular Season,(2020-21),Team Stats,NaN,Boston Celtics,41.5,88.9,0.47,13.6,...,0.78,10.6,33.6,44.3,23.5,7.7,5.3,14.1,20.4,112.6
BRO,NBA,Regular Season,(2020-21),Team Stats,NaN,Brooklyn Nets,43.1,87.3,0.49,14.2,...,0.80,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6
CHA,NBA,Regular Season,(2020-21),Team Stats,NaN,Charlotte Hornets,39.9,87.8,0.45,13.7,...,0.76,10.6,33.2,43.8,26.8,7.8,4.8,14.8,18.0,109.5
CHI,NBA,Regular Season,(2020-21),Team Stats,NaN,Chicago Bulls,42.2,88.6,0.48,12.6,...,0.79,9.6,35.3,45.0,26.8,6.7,4.2,15.1,18.9,110.7


In [23]:
# Create UI for our prediction model
root = tk.Tk()
root.title('NBA Predictor')
root.geometry("554x380")
#root.configure(bg='white')

# Creat labels
# head labels
logo = PhotoImage(file= 'imgs/p.png')
playoffimg = Label(root, image = logo, anchor="center")
playoffimg.grid(row=1,columnspan=3)

head = Label(root, text = "2020-21 NBA Playoffs Game Result Predictor", font = ('Helvetica',20),
            fg = 'blue', relief = tk.RAISED, borderwidth = 3, background = 'white')
head.grid(row=0,column=0,columnspan=3)

row6 = Label(root, text = " ", font = ('comic sans',20))
row6.grid(row=7,column=0,columnspan=2) 

# team labels
awayteamlabel = Label(root, text = "Away", font = ('comic sans',20, 'bold'), fg='red')
awayteamlabel.grid(row=2,column=0,columnspan=1)             
hometeamlabel = Label(root, text = "Home", font = ('comic sans',20, 'bold'), fg='blue')
hometeamlabel.grid(row=2,column=1,columnspan=1) 


# series score and betting line labels
ssbl = Label(root, text = "Money Line", font = ('comic sans',20, 'bold'), fg = 'green')
ssbl.grid(row=2,column=2,columnspan=1) 
#seriesscore = ttk.Combobox(root,font = ('comic sans',20),width=8)
#seriesscore['value'] = ('0-0','0-1','0-2','0-3','1-0','1-1','1-2','1-3',
                     #'2-0','2-1','2-2','2-3','3-0','3-1','3-2','3-3')
#seriesscore.grid(row=5,column=0,padx=5,pady=5,columnspan=1)

# user inputs
hometeam_var = ttk.Combobox(root,font = ('comic sans',20),width=8)
hometeam_var['value'] = ('ATL','BOS','BRO','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHO','POR','UTA','WAS')
hometeam_var.grid(row=3,column=1,padx=5,pady=5,columnspan=1)
awayteam_var = ttk.Combobox(root,font = ('comic sans',20),width=8)
awayteam_var['value'] = ('ATL','BOS','BRO','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHO','POR','UTA','WAS')
awayteam_var.grid(row=3,column=0,padx=5,pady=5,columnspan=1)

userin = IntVar()
bettingline= Entry(root,textvariable=userin,font = ('comic sans',20),width=9)
bettingline.grid(row=3,column=2,padx=5,pady=5,columnspan=1)

# def function to make prediction
def action():
#     global DB
    import pandas as pd
    hometeam = hometeam_var.get()
    awayteam = awayteam_var.get()
    bettingline = userin.get()
    predictor_df = pd.DataFrame(columns=['FGM','FGA','FG%','3FGM',
                                         '3FGA','3FG%','FTM','FTA','FT%','OFF','DEF','REB','AST',
                                         'STL','BLK','TO','PF','PTS','OPFGM','OPFGA','OPFG%',
                                         'OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%','OPOFF',
                                         'OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF','OPPTS','H Betting Line'])

    
    FGM=df_team_2021.loc[awayteam,'FGM']
    predictor_df.loc[0,'FGM']=FGM
    
    FGA=df_team_2021.loc[awayteam,'FGA']
    predictor_df.loc[0,'FGA']=FGA
    
    FGP=df_team_2021.loc[awayteam,'FG%']
    predictor_df.loc[0,'FG%']=FGP
    
    TFGM=df_team_2021.loc[awayteam,'3FGM']
    predictor_df.loc[0,'3FGM']=TFGM
    
    TFGMA=df_team_2021.loc[awayteam,'3FGA']
    predictor_df.loc[0,'3FGA']=TFGMA
    
    TFGP=df_team_2021.loc[awayteam,'3FG%']
    predictor_df.loc[0,'3FG%']=TFGP
    
    FTM=df_team_2021.loc[awayteam,'FTM']
    predictor_df.loc[0,'FTM']=FTM
    
    FTA=df_team_2021.loc[awayteam,'FTA']
    predictor_df.loc[0,'FTA']=FTA
    
    FTP=df_team_2021.loc[awayteam,'FT%']
    predictor_df.loc[0,'FT%']=FTP
    
    OFF=df_team_2021.loc[awayteam,'OFF']
    predictor_df.loc[0,'OFF']=OFF
    
    DEF=df_team_2021.loc[awayteam,'DEF']
    predictor_df.loc[0,'DEF']=DEF
    
    REB=df_team_2021.loc[awayteam,'REB']
    predictor_df.loc[0,'REB']=REB
    
    AST=df_team_2021.loc[awayteam,'AST']
    predictor_df.loc[0,'AST']=AST
    
    STL=df_team_2021.loc[awayteam,'STL']
    predictor_df.loc[0,'STL']=STL
    
    BLK=df_team_2021.loc[awayteam,'BLK']
    predictor_df.loc[0,'BLK']=BLK
    
    TO=df_team_2021.loc[awayteam,'TO']
    predictor_df.loc[0,'TO']=TO
    
    PF=df_team_2021.loc[awayteam,'PF']
    predictor_df.loc[0,'PF']=PF
    
    PTS=df_team_2021.loc[awayteam,'PTS']
    predictor_df.loc[0,'PTS']=PTS
    
    OPFGM=df_team_2021.loc[hometeam,'FGM']
    predictor_df.loc[0,'OPFGM']=OPFGM
    
    OPFGA=df_team_2021.loc[hometeam,'FGA']
    predictor_df.loc[0,'OPFGA']=OPFGA
    
    OPFGP=df_team_2021.loc[hometeam,'FG%']
    predictor_df.loc[0,'OPFG%']=OPFGP
    
    OP3FGM=df_team_2021.loc[hometeam,'3FGM']
    predictor_df.loc[0,'OP3FGM']=OP3FGM
    
    OP3FGMA=df_team_2021.loc[hometeam,'3FGA']
    predictor_df.loc[0,'OP3FGA']=OP3FGMA
    
    OP3FGP=df_team_2021.loc[hometeam,'3FG%']
    predictor_df.loc[0,'OP3FG%']=OP3FGP
    
    OPFTM=df_team_2021.loc[hometeam,'FTM']
    predictor_df.loc[0,'OPFTM']=OPFTM
    
    OPFTA=df_team_2021.loc[hometeam,'FTA']
    predictor_df.loc[0,'OPFTA']=OPFTA
    
    OPFTP=df_team_2021.loc[hometeam,'FT%']
    predictor_df.loc[0,'OPFT%']=OPFTP
    
    OPOFF=df_team_2021.loc[hometeam,'OFF']
    predictor_df.loc[0,'OPOFF']=OPOFF
    
    OPDEF=df_team_2021.loc[hometeam,'DEF']
    predictor_df.loc[0,'OPDEF']=OPDEF
    
    OPREB=df_team_2021.loc[hometeam,'REB']
    predictor_df.loc[0,'OPREB']=OPREB

    OPAST=df_team_2021.loc[hometeam,'AST']
    predictor_df.loc[0,'OPAST']=OPAST

    OPSTL=df_team_2021.loc[hometeam,'STL']
    predictor_df.loc[0,'OPSTL']=OPSTL
    
    OPBLK=df_team_2021.loc[hometeam,'BLK']
    predictor_df.loc[0,'OPBLK']=OPBLK
    
    OPTO=df_team_2021.loc[hometeam,'TO']
    predictor_df.loc[0,'OPTO']=OPTO
    
    OPPF=df_team_2021.loc[hometeam,'PF']
    predictor_df.loc[0,'OPPF']=OPPF
    
    OPPTS=df_team_2021.loc[hometeam,'PTS']
    predictor_df.loc[0,'OPPTS']=OPPTS
    
    predictor_df.loc[0,'H Betting Line']=bettingline
    
    output=df_playoffsstats_forest_model.predict(predictor_df)
    if output == 0 :
        result =  f'{awayteam} Will Win',
        teamimg = Image.open(f'imgs/{awayteam}.png')
        teamimg = teamimg.resize((150, 150), Image. ANTIALIAS)
        teamimg = ImageTk.PhotoImage(teamimg)
    else: # output == 1:
        result =  f'{hometeam} Will Win',
        teamimg = Image.open(f'imgs/{hometeam}.png')
#         teamimg = PhotoImage(file = f'imgs/{hometeam}.png')
        teamimg = teamimg.resize((150, 150), Image. ANTIALIAS)
        teamimg = ImageTk.PhotoImage(teamimg)
    return result, teamimg
#result, teamimg = action()

# def function to display the prediction result
def result_display():
    result,teamimg = action()
    result_display = Label(root, text = "Win!!!", font = ('comic sans',32,"bold italic"), fg='gold')#tk.Text(master = root, height = 3, width=20)
    result_display.grid(row=7,column=2)
    #result_display.insert(tk.END, result)
    
    result_display_img = Label(root, image = teamimg, anchor="center")
    result_display_img.grid(row=7,column=1)
    result_display_img.insert(teamimg)

# make prediction button
predict = Button(root, text='Predict', font = ('comic sans',20), command=result_display, background = 'white')
predict.place(x=30, y=260)

# winnerimg = Label(root, image = teamimg, anchor="center")
# winnerimg.grid(row=7,columnspan=3)

root.mainloop()


## Testing other stuff

In [50]:
playoff2018 = get_schedule(2018, playoffs = True)
playoff2017 = get_schedule(2017, playoffs = True)
playoff2016 = get_schedule(2016, playoffs = True)
playoff2015 = get_schedule(2015, playoffs = True)
playoff2014 = get_schedule(2014, playoffs = True)
playoff2013 = get_schedule(2013, playoffs = True)
playoff2012 = get_schedule(2012, playoffs = True)
playoff2011 = get_schedule(2011, playoffs = True)
playoff2010 = get_schedule(2010, playoffs = True)
playoff2009 = get_schedule(2009, playoffs = True)
playoff2008 = get_schedule(2008, playoffs = True)

In [36]:
playoff2009.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
1231,2009-04-18,Chicago Bulls,105,Boston Celtics,103
1232,2009-04-18,Detroit Pistons,84,Cleveland Cavaliers,102
1233,2009-04-18,Dallas Mavericks,105,San Antonio Spurs,97
1234,2009-04-18,Houston Rockets,108,Portland Trail Blazers,81
1235,2009-04-19,Utah Jazz,100,Los Angeles Lakers,113


In [149]:
playoff_list= [playoff2008,playoff2009,playoff2010,playoff2011,playoff2012,playoff2013,playoff2014,
               playoff2015,playoff2016,playoff2017,playoff2018,playoff2019,playoff2020]
playoffsdata= pd.concat(playoff_list, join = "inner")
playoffsdata.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
1231,2008-04-19,Washington Wizards,86,Cleveland Cavaliers,93
1232,2008-04-19,Phoenix Suns,115,San Antonio Spurs,117
1233,2008-04-19,Dallas Mavericks,92,New Orleans Hornets,104
1234,2008-04-19,Utah Jazz,93,Houston Rockets,82
1235,2008-04-20,Toronto Raptors,100,Orlando Magic,114


In [95]:
np.savetxt('playoffsdata.csv', playoffsdata, delimiter =",", fmt ="% s")

In [150]:
df_updatedmain_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line']

In [37]:
csv_updatedmain = Path('updatedmain.csv')
df_updatedmain = pd.read_csv(csv_updatedmain)
df_updatedmain.head()

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,Playoffs,2008-04-19,(2007-08),WAS,CLE,86,93,7,36.41,81.65,...,13.32,31.26,44.57,20.00,7.06,5.21,13.96,20.88,96.38,-175
1,Playoffs,2008-04-19,(2007-08),PHO,SAS,115,117,2,41.37,82.71,...,9.40,31.85,41.26,20.95,6.35,4.11,12.62,18.74,95.37,-200
2,Playoffs,2008-04-19,(2007-08),DAL,NOH,92,104,12,36.89,79.45,...,11.43,30.41,41.84,21.77,7.77,3.91,11.88,18.67,100.85,-205
3,Playoffs,2008-04-19,(2007-08),UTA,HOU,93,82,-11,39.99,80.39,...,12.22,32.43,44.65,21.43,7.29,5.12,13.72,19.62,96.72,-110
4,Playoffs,2008-04-20,(2007-08),TOR,ORL,100,114,14,38.39,82.02,...,9.37,32.65,42.01,20.80,6.34,4.06,14.30,20.59,104.48,-265


In [38]:
# Make a result column, set home team win to 1, away team win to 0.
results = []
def mr(df_updatedmain):
    if df_updatedmain['Mar.'] > 0 :
        return 1
    else:
        return 0
df_updatedmain['results'] = df_updatedmain.apply(mr,axis=1)

df_updatedmain.head()

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,results
0,Playoffs,2008-04-19,(2007-08),WAS,CLE,86,93,7,36.41,81.65,...,31.26,44.57,20.00,7.06,5.21,13.96,20.88,96.38,-175,1
1,Playoffs,2008-04-19,(2007-08),PHO,SAS,115,117,2,41.37,82.71,...,31.85,41.26,20.95,6.35,4.11,12.62,18.74,95.37,-200,1
2,Playoffs,2008-04-19,(2007-08),DAL,NOH,92,104,12,36.89,79.45,...,30.41,41.84,21.77,7.77,3.91,11.88,18.67,100.85,-205,1
3,Playoffs,2008-04-19,(2007-08),UTA,HOU,93,82,-11,39.99,80.39,...,32.43,44.65,21.43,7.29,5.12,13.72,19.62,96.72,-110,0
4,Playoffs,2008-04-20,(2007-08),TOR,ORL,100,114,14,38.39,82.02,...,32.65,42.01,20.80,6.34,4.06,14.30,20.59,104.48,-265,1


In [193]:
# Construct the X_train and y_train datasets
df_updatedmain_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_updatedmain_y_train = df_updatedmain['results']

In [194]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_updatedmain_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_updatedmain_forest_model.fit(df_updatedmain_x_train, df_updatedmain_y_train)

RandomForestClassifier(max_depth=3, random_state=0)

In [195]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_updatedmain_forest_model.predict(df_2021_test_X)

In [ ]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results_updatedmain = df_2021_test_y.to_frame()
playoff2021_results_updatedmain["Predicted Mar."] = playoff2021_predictions_updatedmain
playoff2021_results_updatedmain["Ori Predicted Mar."] = playoff2021_predictions
playoff2021_results_updatedmain.head(40)

In [197]:
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain))

0.5882352941176471


In [76]:
# load 2008-2020 playoffs data
csv_updatedmain = Path('updatedmain.csv')
df_playoffsstats = pd.read_csv(csv_updatedmain)
# Make a result column, set home team win to 1, away team win to 0.
results = []
def mr(df_playoffsstats):
    if df_playoffsstats['Mar.'] > 0 :
        return 1
    else:
        return 0
df_playoffsstats['results'] = df_playoffsstats.apply(mr,axis=1)

df_playoffsstats.head(5)

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,results
0,Playoffs,2008-04-19,(2007-08),WAS,CLE,86,93,7,36.41,81.65,...,31.26,44.57,20.00,7.06,5.21,13.96,20.88,96.38,-175,1
1,Playoffs,2008-04-19,(2007-08),PHO,SAS,115,117,2,41.37,82.71,...,31.85,41.26,20.95,6.35,4.11,12.62,18.74,95.37,-200,1
2,Playoffs,2008-04-19,(2007-08),DAL,NOH,92,104,12,36.89,79.45,...,30.41,41.84,21.77,7.77,3.91,11.88,18.67,100.85,-205,1
3,Playoffs,2008-04-19,(2007-08),UTA,HOU,93,82,-11,39.99,80.39,...,32.43,44.65,21.43,7.29,5.12,13.72,19.62,96.72,-110,0
4,Playoffs,2008-04-20,(2007-08),TOR,ORL,100,114,14,38.39,82.02,...,32.65,42.01,20.80,6.34,4.06,14.30,20.59,104.48,-265,1


In [77]:
df_corr = df_playoffsstats.drop(columns = ['Vscore','Hscore', 'Mar.'])
playoff2021_predictions_updatedmain_corr = df_corr.corr().abs()
playoff2021_predictions_updatedmain_corr = playoff2021_predictions_updatedmain_corr.loc[playoff2021_predictions_updatedmain_corr['results']>0.05]
playoff2021_predictions_updatedmain_corr.index                                                                                       

Index(['FGM', 'FG%', '3FGM', '3FGA', '3FG%', 'FT%', 'OFF', 'DEF', 'REB', 'AST',
       'BLK', 'TO', 'PTS', 'OPFGA', 'OPFG%', 'OP3FG%', 'OPOFF', 'OPREB',
       'OPSTL', 'OPBLK', 'OPTO', 'H Betting Line', 'results'],
      dtype='object')

In [78]:
df_playoffsstats = df_playoffsstats[['FGM', 'FG%', '3FGM', '3FGA', '3FG%', 'FT%', 'OFF', 'DEF', 'REB', 'AST',
       'BLK', 'TO', 'PTS', 'OPFGA', 'OPFG%', 'OP3FG%', 'OPOFF', 'OPREB',
       'OPSTL', 'OPBLK', 'OPTO', 'H Betting Line', 'results']]

In [81]:
df_2021_test_X_test = df_2021_test_X[['FGM', 'FG%', '3FGM', '3FGA', '3FG%', 'FT%', 'OFF', 'DEF', 'REB', 'AST',
       'BLK', 'TO', 'PTS', 'OPFGA', 'OPFG%', 'OP3FG%', 'OPOFF', 'OPREB',
       'OPSTL', 'OPBLK', 'OPTO', 'H Betting Line']]

In [83]:
df_updatedmain_x_train = df_playoffsstats[['FGM', 'FG%', '3FGM', '3FGA', '3FG%', 'FT%', 'OFF', 'DEF', 'REB', 'AST',
       'BLK', 'TO', 'PTS', 'OPFGA', 'OPFG%', 'OP3FG%', 'OPOFF', 'OPREB',
       'OPSTL', 'OPBLK', 'OPTO', 'H Betting Line']]
df_updatedmain_y_train = df_playoffsstats['results']

In [113]:
df_updatedmain_forest_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
df_updatedmain_forest_model.fit(df_updatedmain_x_train, df_updatedmain_y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [114]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_updatedmain_forest_model.predict(df_2021_test_X_test)
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain))

0.5569620253164557


In [236]:
playoff2021_predictions_updatedmain_corr.sort_values('results')

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,results
H Betting Line,0.178478,0.006590,0.309231,0.139936,0.111286,0.216288,-0.019594,-0.020918,0.019392,-0.116820,...,-0.112098,-0.062804,-0.046121,-0.088907,-0.177202,-0.017614,0.086648,-0.121134,1.000000,-0.174111
FG%,0.520859,-0.050674,1.000000,0.105844,0.026624,0.479720,0.052343,0.045205,0.132749,-0.454827,...,0.073983,0.092583,-0.129041,-0.154345,-0.022321,-0.074432,0.051389,0.070409,0.309231,-0.143659
PTS,0.899025,0.770627,0.423392,0.804916,0.791423,0.235915,0.046999,0.091941,0.283191,-0.209855,...,0.586425,0.483075,-0.121550,0.005763,0.018652,0.032349,0.130480,0.645726,0.173344,-0.127032
FGM,1.000000,0.817907,0.520859,0.614725,0.593789,0.210362,-0.192584,-0.201171,0.275664,-0.174753,...,0.558739,0.456596,-0.082485,-0.025782,0.007681,0.019773,0.095793,0.591384,0.178478,-0.124893
AST,0.752336,0.519499,0.524241,0.445166,0.403943,0.292179,-0.305807,-0.318272,0.171649,-0.318003,...,0.396679,0.356269,-0.121445,-0.049692,-0.015167,0.006828,0.047167,0.384153,0.159987,-0.122756
3FGM,0.614725,0.659218,0.105844,1.000000,0.987806,0.297987,-0.156544,-0.123837,0.122182,-0.350122,...,0.562370,0.444117,-0.080245,0.073067,0.059285,0.047981,0.073857,0.589013,0.139936,-0.119616
3FG%,0.210362,-0.064372,0.479720,0.297987,0.154195,1.000000,-0.015480,-0.150987,0.136498,-0.368349,...,-0.061343,-0.002624,-0.113517,-0.007821,-0.032735,-0.035219,0.044024,-0.040860,0.216288,-0.117413
DEF,0.660169,0.692586,0.141214,0.686161,0.683868,0.149923,-0.171736,-0.125107,0.200244,-0.210698,...,0.507181,0.388465,0.020731,0.105904,0.025304,0.038386,0.036457,0.585308,0.156297,-0.112916
3FGA,0.593789,0.687378,0.026624,0.987806,1.000000,0.154195,-0.153810,-0.097300,0.100521,-0.302437,...,0.588149,0.458131,-0.063370,0.080446,0.066413,0.055643,0.069717,0.613366,0.111286,-0.104223
FT%,0.275664,0.225972,0.132749,0.122182,0.100521,0.136498,0.067339,-0.213768,1.000000,-0.225604,...,0.180225,0.113328,-0.066943,0.010687,-0.059071,0.034466,-0.055737,0.231318,0.019392,-0.080407


In [25]:
import math
def win_probs(home_elo,away_elo,home_court_advantage):
    h = math.pow(10,home_elo/400)
    r = math.pow(10,away_elo/400)
    a = math.pow(10,home_court_advantage/400)
    
    denom = r+a*h
    home_prob = a*h/denom
    away_prob = a/denom
    
    return home_prob, away_prob

def home_odds_on(home_elo,away_elo,home_court_advantage):
    h = math.pow(10,home_elo/400)
    r = math.pow(10,away_elo/400)
    a = math.pow(10,home_court_advantage/400)
    return a*h/r

def elo_k(MOV,elo_diff):
    k=20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier
def update_elo(Hscore, Vscore, home_elo, away_elo, home_court_advantage):
    home_prob,away_prob=win_probs(home_elo,away_elo,home_court_advantage)
    
    if (Hscore - Vscore >0):
        home_win =1
        away_win = 0
    else:
        home_win = 0
        away_win =1
    k = elo_k(h_score - a_score, home_elo -away_elo)
    updated_home_elo = home_elo + k*(home_win - home_prob)
    updated_away_elo = away_elo + k*(away_win - away_prob)
    return updated_home_elo, updated_away_elo

def get_prev_elo(team,date,season,team_stats,elo_df):
    prev_game = team_stats[team_stats['Season'] < game_date][(team_stats['HTeam'] == team) | (team_stats['VTeam'] == team)].sort_values(by = 'Season').tail(1).iloc[0] 

    if team == prev_game['HTeam'] :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['HTeam_Elo_After'].values[0]
    else :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['VTeam_Elo_After'].values[0]
  
    if prev_game['League'] != season :
        return (0.75 * elo_rating) + (0.25 * 1505)
    else :
        return elo_rating

In [26]:
elo_rating.head(50)

NameError: name 'elo_rating' is not defined

In [76]:
csv_updatedmain = Path('test.csv')
df_updatedmain = pd.read_csv(csv_updatedmain)

df_updatedmain.head(1)

,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,FGA,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,Playoffs,4/19/08 12:00 AM,(2007-08),WAS,CLE,86,93,7,36.41,81.65,...,13.32,31.26,44.57,20.0,7.06,5.21,13.96,20.88,96.38,-175


In [77]:
df_updatedmain.reset_index(inplace=True)
df_updatedmain['Game_ID'] = 1000+df_updatedmain['index']
df_updatedmain['Date'] = pd.to_datetime(df_updatedmain.Date)
df_updatedmain.tail(1)

,index,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,FGM,...,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,Game_ID
1084,1084,Finals,2020-10-11,(2019-20),LAL,MIA,106,93,-13,42.3,...,35.88,44.37,25.92,7.49,4.55,14.9,20.56,112.04,210,2084


In [78]:
df_updatedmain.sort_values(by = 'Date', inplace = True)
df_updatedmain.reset_index(inplace=True, drop = True)
elo_df = pd.DataFrame(columns=['Game_ID', 'HTeam', 'VTeam', 'HTeam_Elo_Before', 'VTeam_Elo_Before', 'HTeam_Elo_After', 'VTeam_Elo_After'])
teams_elo_df = pd.DataFrame(columns=['Game_ID','Team', 'Elo', 'Date', 'Where_Played', 'Season']) 

for index, row in df_updatedmain.iterrows(): 
    game_id = row['Game_ID']
    game_date = row['Date']
    season = row['Season']
    h_team, a_team = row['HTeam'], row['VTeam']
    h_score, a_score = row['Hscore'], row['Vscore'] 

    if (h_team not in elo_df['HTeam'].values and h_team not in elo_df['VTeam'].values) :
        h_team_elo_before = 1500
    else :
        h_team_elo_before = get_prev_elo(h_team, game_date, season, df_updatedmain, elo_df)

    if (a_team not in elo_df['HTeam'].values and a_team not in elo_df['VTeam'].values) :
        a_team_elo_before = 1500
    else :
        a_team_elo_before = get_prev_elo(a_team, game_date, season, df_updatedmain, elo_df)

        h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)

    new_row = {'Game_ID': game_id, 'HTeam': h_team, 'VTeam': a_team, 'HTeam_Elo_Before': h_team_elo_before, 'VTeam_Elo_Before': a_team_elo_before, \
                                                                        'HTeam_Elo_After' : h_team_elo_after, 'VTeam_Elo_After': a_team_elo_after}
#   teams_row_one = {'Game_ID': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Date': game_date, 'Where_Played': 'Home', 'League': season}
#   teams_row_two = {'Game_ID': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Date': game_date, 'Where_Played': 'Away', 'League': season}
  
    elo_df = elo_df.append(new_row, ignore_index = True)
#   teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
#   teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)

TypeError: '<' not supported between instances of 'str' and 'Timestamp'

In [71]:
season_teams_stats = df_updatedmain.merge(elo_df.drop(columns = ['HTeam', 'VTeam']), on = 'Game_ID')
season_teams_stats.head()

,level_0,index,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,...,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,Game_ID,HTeam_Elo_Before,VTeam_Elo_Before,HTeam_Elo_After,VTeam_Elo_After
0,1084,1084,Finals,2020-10-11,(2019-20),LAL,MIA,106,93,-13,...,4.55,14.9,20.56,112.04,210,2084,1500,1500,1485.345335,1524.502959


In [54]:
season_teams_stats.head(50)

,level_0,index,Type,Date,Season,VTeam,HTeam,Vscore,Hscore,Mar.,...,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,Game_ID,HTeam_Elo_Before,VTeam_Elo_Before,HTeam_Elo_After,VTeam_Elo_After
0,1084,1084,Finals,2020-10-11,(2019-20),LAL,MIA,106,93,-13,...,4.55,14.9,20.56,112.04,210,2084,1500,1500,1485.345335,1524.502959


In [ ]:
# #teams_elo_df.set_index(["Team"], append=True)
# #dataset = teams_elo_df.pivot(index="Team",values="Elo", columns="Date")
# dates = list(set([d.strptime("%m-%d-%Y") for d in teams_elo_df["Date"]]))
# dates = sorted(dates, key=lambda x: time.strptime(x, '%m-%d-%Y'))
# teams = team_stats["VTeam"]
# dataset = pd.DataFrame(columns=dates)
# dataset["Team"] = teams.drop_duplicates()
# dataset = dataset.set_index("Team")
# for index, row in teams_elo_df.iterrows():
#   date = row["Date"].strftime("%m-%d-%Y")
#   team = row["Team"]
#   elo = row["Elo"]
#   dataset[date][team] = elo

# teams_elo_df['Elo'] = teams_elo_df['Elo'].astype(float)

# elo_df

In [438]:
team_season_stats_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line','HTeam_Elo_Before','VTeam_Elo_Before','HTeam_Elo_After',
                            'VTeam_Elo_After']

In [439]:
# Make a result column, set home team win to 1, away team win to 0.
results = []
def mr(season_teams_stats):
    if season_teams_stats['Mar.'] > 0 :
        return 1
    else:
        return 0
season_teams_stats['results'] = season_teams_stats.apply(mr,axis=1)

season_teams_stats.head()

,level_0,index,Type,Season,League,VTeam,HTeam,Vscore,Hscore,Mar.,...,OPTO,OPPF,OPPTS,H Betting Line,Game_ID,HTeam_Elo_Before,VTeam_Elo_Before,HTeam_Elo_After,VTeam_Elo_After,results
0,0,0,Playoffs,2008-04-19,(2007-08),WAS,CLE,86,93,7,...,13.96,20.88,96.38,-175,1000,1500,1500,1506.763631,1499.998211,1
1,1,1,Playoffs,2008-04-19,(2007-08),PHO,SAS,115,117,2,...,12.62,18.74,95.37,-200,1001,1500,1500,1503.884686,1499.998972,1
2,2,2,Playoffs,2008-04-19,(2007-08),DAL,NOH,92,104,12,...,11.88,18.67,100.85,-205,1002,1500,1500,1509.355197,1499.997525,1
3,3,3,Playoffs,2008-04-19,(2007-08),UTA,HOU,93,82,-11,...,13.72,19.62,96.72,-110,1003,1500,1500,1486.830104,1522.020388,0
4,4,4,Playoffs,2008-04-20,(2007-08),TOR,ORL,100,114,14,...,14.30,20.59,104.48,-265,1004,1500,1500,1510.340441,1499.997264,1


In [206]:
# Construct the X_train and y_train datasets
df_updatedmain_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_updatedmain_y_train = df_updatedmain['results']
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_updatedmain_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_updatedmain_forest_model.fit(df_updatedmain_x_train, df_updatedmain_y_train)
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_updatedmain_forest_model.predict(df_2021_test_X)
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results_updatedmain = df_2021_test_y.to_frame()
playoff2021_results_updatedmain["Predicted Mar."] = playoff2021_predictions_updatedmain
playoff2021_results_updatedmain["Ori Predicted Mar."] = playoff2021_predictions
playoff2021_results_updatedmain.head(5)

,results,Predicted Mar.,Ori Predicted Mar.
Time,,,
5/22/2021 0:00,1,1,1
5/22/2021 0:00,0,1,1
5/22/2021 0:00,1,1,0
5/22/2021 0:00,0,1,1
5/23/2021 0:00,1,1,0


In [160]:
# Construct the X_train and y_train datasets
df_LR_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_LR_y_train = df_updatedmain['results']
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_LR_model = LogisticRegression(random_state = 0)
df_LR_model.fit(df_LR_x_train, df_LR_y_train)
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain_LR = df_LR_model.predict(df_2021_test_X)
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results_updatedmain_LR = df_2021_test_y.to_frame()
playoff2021_results_updatedmain_LR["Predicted Mar."] = playoff2021_predictions_updatedmain_LR
playoff2021_results_updatedmain_LR["Ori Predicted Mar."] = playoff2021_predictions
playoff2021_results_updatedmain_LR.head(5)

C:\Users\william\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,results,Predicted Mar.,Ori Predicted Mar.
Time,,,
2021-05-22,1,1,1
2021-05-22,0,1,1
2021-05-22,1,1,1
2021-05-22,0,1,1
2021-05-23,1,1,1


In [161]:
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain_LR))

0.5949367088607594


In [248]:
# Construct the X_train and y_train datasets
df_NB_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_NB_y_train = df_updatedmain['results']
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_updatedmain_NB_model = GaussianNB()
df_updatedmain_NB_model.fit(df_NB_x_train, df_NB_y_train)
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_updatedmain_NB_model.predict(df_2021_test_X)
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain))

0.45569620253164556


In [219]:
# Construct the X_train and y_train datasets
df_updatedmain_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_updatedmain_y_train = df_updatedmain['results']
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_updatedmain_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_updatedmain_forest_model.fit(df_updatedmain_x_train, df_updatedmain_y_train)
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_updatedmain_forest_model.predict(df_2021_test_X)
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain))

0.5882352941176471


In [158]:
from sklearn.tree import DecisionTreeClassifier

In [245]:
# Construct the X_train and y_train datasets
df_DTC_x_train = df_updatedmain[df_updatedmain_x_var_list]
df_DTC_y_train = df_updatedmain['results']
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_DTC_model = DecisionTreeClassifier(max_depth=4)
df_DTC_model.fit(df_DTC_x_train, df_DTC_y_train)
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions_updatedmain = df_DTC_model.predict(df_2021_test_X)
print(metrics.accuracy_score(df_2021_test_y, playoff2021_predictions_updatedmain))


0.5569620253164557


In [254]:
# load 2008-2020 playoffs data
csv_betting = Path('betting.csv')
df_betting = pd.read_csv(csv_betting)
df_betting.tail(5)

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
2325,630,532,H,LAClippers,29,28,26,20,103,1,214.5,-105,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2326,701,541,V,Atlanta,22,34,22,34,112,219.5,213,165,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2327,701,542,H,Milwaukee,36,29,26,32,123,4,4.5,-185,105.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2328,703,561,V,Milwaukee,28,19,44,27,118,215,219.5,140,111.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2329,703,562,H,Atlanta,24,19,29,35,107,pk,3.5,-160,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [259]:
df1=df_betting[df_betting.index%2==1]
df1.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
1,1222,502,H,Brooklyn,40,23,36,26,125,5.5,7.5,-320,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1222,504,H,LALakers,19,35,24,31,109,2.5,2,-135,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1223,552,H,Cleveland,24,41,28,28,121,1,217,120,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1223,554,H,Indiana,35,26,27,33,121,8.5,7.5,-330,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1223,556,H,Orlando,27,29,23,34,113,220.5,219,170,106.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
